# RAG on FHIR with Knowledge Graphs

### 2. Neo4J & Jupyter Environment
This notebook needs an instance of [Neo4j](https://www.neo4j.com) to talk to. I used docker to run Neo4J locally using the following command:
```
docker run --name testneo4j -p7474:7474 -p7687:7687 -d \
    -v $HOME/neo4j/data:/data \
    -v $HOME/neo4j/logs:/logs \
    -v $HOME/neo4j/import:/var/lib/neo4j/import \
    -v $HOME/neo4j/plugins:/plugins \
    --env NEO4J_AUTH=neo4j/password \
    neo4j:latest
```
**Note:** No particular plugins are needed. 

You can also use a Neo4J Aurora instance. 

#### Jupyter Environment
Regardless of how you run Neo4J. You need to set some environment variables in the notebook's environment:

| Variable | Description | Value for above Docker |
|----------|-------------|------------------------|
| NEO4J_URL | Where to find the instance of Neo4j. | bolt://localhost:7687 |
| NEO4J_USER | The username for the database. | neo4j |
| NEO4J_PASSWORD | The password for the database. | password |


### 3. Synthetic data and working directory
The data I used for this notebook came from [Synthea](https://synthea.mitre.org/). Using the 

All the questions here us the FHIR Bundle: `fhir_data/stanfor_llm_on_fhir`

In [54]:
# Imports needed

import glob
import json
import os
import re

## Establish Database Connection

The cell connects to the Neo4J instance. It relies on several environment variables. 

**PLEASE NOTE**: The variable have been changed to support multiple databases in the same instance. 

| Variable            | Description                          | Sample Value          |
|---------------------|--------------------------------------|-----------------------|
| FHIR_GRAPH_URL      | Where to find the instance of Neo4j. | bolt://localhost:7687 |
| FHIR_GRAPH_USER     | The username for the database.       | neo4j                 |
| FHIR_GRAPH_PASSWORD | The password for the database.       | password              |
| FHIR_GRAPH_DATABASE | The name of the database instance.   | neo4j                 |

In [55]:
NEO4J_URI = os.getenv('FHIR_GRAPH_URL', 'neo4j://localhost:7687')
USERNAME = os.getenv('FHIR_GRAPH_USER', 'neo4j')
PASSWORD = os.getenv('FHIR_GRAPH_PASSWORD', 'password')
DATABASE = os.getenv('FHIR_GRAPH_DATABASE', 'neo4j')

## Helper Database Cells

The following three cells are here to be used to manage the database. They do not need to be run on a blank database. 

http://localhost:7474/browser/

### Create Vector Index 

This cell creates a new vector index, using the index created above. 

This is here because running the cell above can take time and only should be done one time when the DB is created. 

In [56]:
from langchain.vectorstores.neo4j_vector import Neo4jVector
from langchain_community.embeddings import HuggingFaceBgeEmbeddings

vector_index = Neo4jVector.from_existing_index(
    HuggingFaceBgeEmbeddings(model_name="BAAI/bge-small-en-v1.5"),
    url=NEO4J_URI,
    username=USERNAME,
    password=PASSWORD,
    database=DATABASE,
    index_name='fhir_text'
)

In [57]:
from questions import stanford_llm_on_fhir_questions

question = stanford_llm_on_fhir_questions["Q2"]
question

ImportError: cannot import name 'stanford_llm_on_fhir_questions' from 'questions' (/home/baptvit/Documents/github/mestrado/fhir-rag/fhir_rag/questions/__init__.py)

In [ ]:
question = "Do I have allergi of Aspirin ?"

In [ ]:
response = vector_index.similarity_search(question, k=50) # k_nearest is not used here because we don't have a retrieval query yet.

### Similary search will be better when we need a feacture extraction

In [ ]:
response = vector_index.similarity_search_with_score(query=question, k=50, score_threshold=0.80) 

### Hibryd search with key name

In [58]:
response = vector_index.similarity_search_with_score(query=question, k=100, score_threshold=0.80, filter={"name": "AllergyIntolerance"}) 

In [59]:
import json 

def resources_count(response: list) -> None:
    output_len = len(response)
    print(f"len return {output_len}")
    
    list_output = []
    output_str = '' 
    for i in range(output_len):
        try:
            output_str += response[i][0].page_content
    
            list_output.append(json.loads(response[i][0].page_content)["resourceType"])
        except:
            print(response[i][0].page_content)
    
    list(set(list_output))
    
    dictionary = {}
    for item in list_output:
        dictionary[item] = dictionary.get(item, 0) + 1
    
    print(dictionary)
    return output_str

output_str = resources_count(response)

len return 9
{'AllergyIntolerance': 9}


## Count the amount of tokens in the output

Here we are interesting in see the amout of tokens of the output and analise wheater the will fit in the provided context window.


In [60]:
import tiktoken

def num_tokens_from_string(string: str, encoding_name: str) -> int:
    encoding = tiktoken.get_encoding(encoding_name)
    num_tokens = len(encoding.encode(string))
    return num_tokens

In [61]:
print(num_tokens_from_string(output_str, "cl100k_base"))

2850


### Graph Search with similairy Search and Hibridy

By adding a retrival_query in chyper langangue we can apply a neaghor retrivel base on a algorithm after the similarity search

Default query

In [66]:
contextualize_query = """
WHERE score >= 0.82
MATCH (node)
RETURN node.text AS text, score, {} AS metadata
"""

Tryng to apply a Dict hyerach

In [67]:
contextualized_vectorstore = Neo4jVector.from_existing_index(
    HuggingFaceBgeEmbeddings(model_name="BAAI/bge-small-en-v1.5"),
    url=NEO4J_URI,
    username=USERNAME,
    password=PASSWORD,
    database=DATABASE,
    index_name='fhir_text',
    retrieval_query=contextualize_query
)

In [68]:
response = contextualized_vectorstore.similarity_search_with_score(query=question, k=100, score_threshold=0.80, filter={"name": "AllergyIntolerance"}) 

In [69]:
response = contextualized_vectorstore.similarity_search_with_score(query="Condition", k=100, score_threshold=0.80) 

In [70]:
response = contextualized_vectorstore.similarity_search_with_score(query="What are my current medications and how should I be taking them?", k=100, score_threshold=0.80) 

In [71]:
len(response)

8

In [72]:
print(response[1][0].page_content)

{"resourceType": "MedicationRequest", "id": "6948c88e-d15c-3f11-a635-be707cb15632", "meta": {"profile": ["http://hl7.org/fhir/us/core/StructureDefinition/us-core-medicationrequest"]}, "status": "stopped", "intent": "order", "category": [{"coding": [{"system": "http://terminology.hl7.org/CodeSystem/medicationrequest-category", "code": "community", "display": "Community"}], "text": "Community"}], "medicationCodeableConcept": {"coding": [{"system": "http://www.nlm.nih.gov/research/umls/rxnorm", "code": "308192", "display": "Amoxicillin 500 MG Oral Tablet"}], "text": "Amoxicillin 500 MG Oral Tablet"}, "subject": {"reference": "urn:uuid:5b3645de-a2d0-d016-0839-bab3757c4c58"}, "encounter": {"reference": "urn:uuid:cf60427f-6701-abf2-82fc-945837df2152"}, "authoredOn": "2016-11-23T13:37:42+00:00", "requester": {"reference": "Practitioner?identifier=http://hl7.org/fhir/sid/us-npi|9999990697", "display": "Dr. Alvin56 Crona259"}, "dosageInstruction": [{"sequence": 1, "text": "Take at regular inter

# Set prompt system



In [73]:
from langchain import PromptTemplate

simple_sys_prompt ='''
You are a highly skilled FHIR Specialist tasked with thoroughly analyzing patient queries. Your goal is to provide comprehensive, detailed responses based on the full context provided within triple quotes.

When answering, ensure your responses are rich in specific details such as locations, costs, timelines, and any other relevant information that would be valuable to the patient. Consider the broader implications and include any additional insights that may assist the patient in making informed decisions.

Before responding, always analyze the entire context to ensure your answer is both accurate and exhaustive. Aim for clarity and depth in your explanations.
|
Context in FHIR R4: """{context}""

Human: {question}
'''

prompt = PromptTemplate.from_template(simple_sys_prompt)

# Config langFuse

In [74]:
from langfuse import Langfuse

langfuse = Langfuse(
  secret_key="",
  public_key="",
  host="https://us.cloud.langfuse.com"
)


from langfuse.callback import CallbackHandler
langfuse_handler = CallbackHandler(
    secret_key="",
  public_key="",
  host="https://us.cloud.langfuse.com"
)
 

## Gemini PRO 1.5

In [75]:
from langchain_google_vertexai import VertexAI
from langchain_google_vertexai import ChatVertexAI
from langchain.chains import RetrievalQA
from IPython.display import display, Markdown
from langfuse.decorators import observe

observe()
def call(query, k=10):
    vector_qa_vertex = RetrievalQA.from_chain_type(
        llm=ChatVertexAI(model="gemini-1.5-pro-002", temperature=0), chain_type="stuff", retriever=contextualized_vectorstore.as_retriever(search_kwargs={'k': k}), # k_nearest is not used here because we don't have a retrieval query yet.
        verbose=True, chain_type_kwargs={"verbose": True, "prompt": prompt}
    )

    return vector_qa_vertex.invoke(query, config={"callbacks": [langfuse_handler]})

In [76]:
response = call("What are my current medications and how should I be taking them?")



> Entering new RetrievalQA chain...


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

You are a highly skilled FHIR Specialist tasked with thoroughly analyzing patient queries. Your goal is to provide comprehensive, detailed responses based on the full context provided within triple quotes.

When answering, ensure your responses are rich in specific details such as locations, costs, timelines, and any other relevant information that would be valuable to the patient. Consider the broader implications and include any additional insights that may assist the patient in making informed decisions.

Before responding, always analyze the entire context to ensure your answer is both accurate and exhaustive. Aim for clarity and depth in your explanations.
|
Context in FHIR R4: """{"resourceType": "MedicationRequest", "id": "3cf09561-5d36-5abd-61dd-7596bea2c5ac", "meta": {"profile": ["http://hl7.org/fhir/us/core/StructureDefinition/us-cor

In [77]:
display(Markdown(response["result"]))

You currently have two active medications prescribed:

* **Fexofenadine hydrochloride 30 MG Oral Tablet:** This is prescribed to be taken *as needed*.  The provided data doesn't specify the exact dosage when taken, so it's crucial to consult the instructions provided on the medication packaging or contact your prescribing physician, Dr. Alvin56 Crona259, for clarification.

* **NDA020800 0.3 ML Epinephrine 1 MG/ML Auto-Injector:** This is also prescribed to be taken *as needed*.  This is likely an EpiPen for allergic reactions.  Again, review the instructions provided with the auto-injector or consult Dr. Crona259 for specific usage instructions.  Given that you have a documented allergy to Aspirin, ensure you understand when and how to use this auto-injector in case of an allergic reaction.

It's important to note that this information is based on the provided data and may not reflect recent changes or discontinued prescriptions.  Always consult with your physician or pharmacist for the most up-to-date information regarding your medications.  They can provide personalized advice and address any questions or concerns you may have.  Don't hesitate to reach out to Dr. Crona259 for clarification on your medication regimen.


## Conditions Summarization

In [ ]:
response = call("Summarize my conditions in details", 50)

In [ ]:
display(Markdown(response["result"]))

## Conditions Summarization - Active

In [ ]:
response = call("Summarize my active conditions", 50)

In [ ]:
display(Markdown(response["result"]))

# Ollama with local models

In [ ]:
from langchain.llms import Ollama
from langchain.chat_models import ChatOllama
from IPython.display import display, Markdown

In [ ]:
ollama_model = 'llama3-gradient' # mistral, orca-mini, llama2, llama3-gradient

In [ ]:
vector_qa = RetrievalQA.from_chain_type(
    llm=ChatOllama(model=ollama_model), chain_type="stuff", retriever=contextualized_vectorstore.as_retriever(search_kwargs={'k': 100}), # k_nearest is not used here because we don't have a retrieval query yet.
    verbose=True, chain_type_kwargs={"verbose": True, "prompt": prompt}
)

In [ ]:
response = vector_qa.run("Whatare my current medications and how should I be taking them?")

In [ ]:
from IPython.display import display, Markdown

In [ ]:
display(Markdown(response))